**Load the model from HuggingFace**

In [1]:
from transformers import AutoProcessor, BitsAndBytesConfig, LlavaNextForConditionalGeneration
import torch
from datasets import load_dataset
from pprint import pprint

**<span style="color:orange">Load the model</span>**

- **<span style="color:yellow">Quantized model: Requires near `8` GB of GPU memory with the configuration provided in the cell below.</span>**
- **<span style="color:yellow">Full model: Requires near `24` GB of GPU for inferencing</span>**

In [2]:
MODEL_ID = "llava-hf/llava-v1.6-mistral-7b-hf"
REPO_ID = "harshareddy21/llava"

In [ ]:
processor = AutoProcessor.from_pretrained(MODEL_ID)

# Define quantization config
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16
)
# Load the base model with adapters on top
model = LlavaNextForConditionalGeneration.from_pretrained(
    REPO_ID,
    torch_dtype=torch.float16,
    quantization_config=quantization_config,
)

adapter_config.json:   0%|          | 0.00/869 [00:00<?, ?B/s]

C:\Projects\My-Projects\Trading-AI\myenv\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Harshavardhan.reddy\.cache\huggingface\hub\models--harshareddy21--llava. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
`low_cpu_mem_usage` was None, now set to True since model is quantized.


**<span style="color:orange">Function for converting the model output to a nicely formatted JSON</span>**

In [ ]:
# let's turn that into JSON
import re
def token2json(tokens, is_inner_value=False, added_vocab=None):
        """
        Convert a (generated) token sequence into an ordered JSON format.
        """
        if added_vocab is None:
            added_vocab = processor.tokenizer.get_added_vocab()

        output = {}

        while tokens:
            start_token = re.search(r"<s_(.*?)>", tokens, re.IGNORECASE)
            if start_token is None:
                break
            key = start_token.group(1)
            key_escaped = re.escape(key)

            end_token = re.search(rf"</s_{key_escaped}>", tokens, re.IGNORECASE)
            start_token = start_token.group()
            if end_token is None:
                tokens = tokens.replace(start_token, "")
            else:
                end_token = end_token.group()
                start_token_escaped = re.escape(start_token)
                end_token_escaped = re.escape(end_token)
                content = re.search(
                    f"{start_token_escaped}(.*?){end_token_escaped}", tokens, re.IGNORECASE | re.DOTALL
                )
                if content is not None:
                    content = content.group(1).strip()
                    if r"<s_" in content and r"</s_" in content:  # non-leaf node
                        value = token2json(content, is_inner_value=True, added_vocab=added_vocab)
                        if value:
                            if len(value) == 1:
                                value = value[0]
                            output[key] = value
                    else:  # leaf nodes
                        output[key] = []
                        for leaf in content.split(r"<sep/>"):
                            leaf = leaf.strip()
                            if leaf in added_vocab and leaf[0] == "<" and leaf[-2:] == "/>":
                                leaf = leaf[1:-2]  # for categorical special tokens
                            output[key].append(leaf)
                        if len(output[key]) == 1:
                            output[key] = output[key][0]

                tokens = tokens[tokens.find(end_token) + len(end_token) :].strip()
                if tokens[:6] == r"<sep/>":  # non-leaf nodes
                    return [output] + token2json(tokens[6:], is_inner_value=True, added_vocab=added_vocab)

        if len(output):
            return [output] if is_inner_value else output
        else:
            return [] if is_inner_value else {"text_sequence": tokens}

**<span style="color:orange">Load the validation dataset</span>**

In [ ]:
dataset = load_dataset("naver-clova-ix/cord-v2", split="validation")

**<span style="color:yellow">Example 1</span>**

In [ ]:
image = dataset[0]["image"]
image

In [ ]:
prompt = f"[INST] <image>\nExtract JSON [/INST]"
max_output_token = 256
inputs = processor(prompt, image, return_tensors="pt").to("cuda:0")
output = model.generate(**inputs, max_new_tokens=max_output_token)
response = processor.decode(output[0], skip_special_tokens=True)
response

**<span style="color:pink">Model's output vs the ground truth</span>**

In [ ]:
import json

generated_json = token2json(response)
print("Expected response:\n") 
pprint(json.loads(dataset[0]["ground_truth"])["gt_parse"])
print()
print("Converting model's output to JSON:\n")
pprint(generated_json)

**<span style="color:yellow">Example 2</span>**

In [ ]:
image = dataset[1]["image"]
prompt = f"[INST] <image>\nExtract JSON [/INST]"
max_output_token = 256
inputs = processor(prompt, image, return_tensors="pt").to("cuda:0")
output = model.generate(**inputs, max_new_tokens=max_output_token)
response = processor.decode(output[0], skip_special_tokens=True)
response

**<span style="color:pink">Model's output vs the ground truth</span>**

In [ ]:
generated_json = token2json(response)
print("Expected response:\n") 
pprint(json.loads(dataset[1]["ground_truth"])["gt_parse"])
print()
print("Converting model's output to JSON:\n")
pprint(generated_json)

**<span style="color:yellow">Example 3</span>**

In [ ]:
image = dataset[10]["image"]
prompt = f"[INST] <image>\nExtract JSON [/INST]"
max_output_token = 256
inputs = processor(prompt, image, return_tensors="pt").to("cuda:0")
output = model.generate(**inputs, max_new_tokens=max_output_token)
response = processor.decode(output[0], skip_special_tokens=True)
response

**<span style="color:pink">Model's output vs the ground truth</span>**

In [ ]:
generated_json = token2json(response)
print("Expected response:\n") 
pprint(json.loads(dataset[10]["ground_truth"])["gt_parse"])
print()
print("Converting model's output to JSON:\n")
pprint(generated_json)